In [ ]:
# Keyframe metadata
combined_keyframes_metadata_filename = None
keyframes_metadata_dir = None

# Object extraction metadata
combined_object_extraction_filename = None
object_extraction_dir = None

# OCR metadata
combined_ocr_metadata_filename = None
ocr_metadata_dir = None

# Tag metadata
combined_tag_metadata_filename = None
tag_metadata_dir = None

# Audio metadata
combined_audio_metadata_filename = None
audio_metadata_dir = None

# # Final metadata
# final_metadata_filename = None

In [ ]:
import os
import json

dir_path = os.getcwd()

In [ ]:
data_extraction_path = f'{dir_path}/data_extraction'
dataset_path = f'{data_extraction_path}/dataset/AIC_Video'

# Keyframe metadata
if not combined_keyframes_metadata_filename:
    combined_keyframes_metadata_filename = 'keyframes_metadata.json'
    
if not keyframes_metadata_dir:
    keyframes_metadata_dir = f'{data_extraction_path}/transnet/keyframes_metadata'
    
# Object extraction metadata
if not combined_object_extraction_filename:
    combined_object_extraction_filename = 'object_extraction_metadata.json'
    
if not object_extraction_dir:
    object_extraction_dir = f'{data_extraction_path}/metadata/object_extraction/object_detection'
    
# OCR metadata
if not combined_ocr_metadata_filename:
    combined_ocr_metadata_filename = 'ocr_metadata.json'
    
if not ocr_metadata_dir:
    ocr_metadata_dir = f'{data_extraction_path}/metadata/ocr'
    
# Tag metadata
if not combined_tag_metadata_filename:
    combined_tag_metadata_filename = 'tag_metadata.json'
    
if not tag_metadata_dir:
    tag_metadata_dir = f'{data_extraction_path}/metadata/tag_output'
    
# Audio metadata
if not combined_audio_metadata_filename:
    combined_audio_metadata_filename = 'audio_metadata.json'
    
if not audio_metadata_dir:
    audio_metadata_dir = f'{data_extraction_path}/audio/audio_recognition'
    
# # final_metadata
# if not final_metadata_filename:
#     final_metadata_filename = 'final_metadata.json'

# Combine keyframe metadata

In [ ]:
def combine_keyframe_metadata_json_files(directory, output_file):
    combined_data = {}

    if os.path.exists(output_file):
        with open(output_file, 'r') as existing_file:
            combined_data = json.load(existing_file)

    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            file_path = os.path.join(directory, filename)
            
            with open(file_path, 'r') as file:
                data = json.load(file)
        change_key_data = {}
        for keyframes_key, keyframes_value in data.items():
            if keyframes_key.split("_")[-1] == "extra":
                change_key_data[f"{keyframes_key.split('_')[0]}_{keyframes_key.split('_')[1]}_{keyframes_key.split('_')[-1]}_{keyframes_key.split('_')[2]}"]= data[keyframes_key]
                
            else:
                change_key_data[keyframes_key] = data[keyframes_key]
        combined_data.update(change_key_data)
        for keyframes_key, keyframes_value in combined_data.items():
            result = {"result": {"video_id": f"{keyframes_value['video_path'].split('/')[-1]}", "frame_id": f"{keyframes_value['frame_index']}"}}
            combined_data[keyframes_key].update(result)
        keys = list(combined_data.keys())
        keys.sort()
        sorted_combined_data = {i : combined_data[i] for i in keys}
    with open(output_file, 'w') as outfile:
        json.dump(sorted_combined_data, outfile)
    
    print(f'Combined keyframe metadata successful: {output_file}')
    

combine_keyframe_metadata_json_files(keyframes_metadata_dir, combined_keyframes_metadata_filename)

Combined keyframe metadata successful: keyframes_metadata.json


# Combine object extraction metadata

In [ ]:
keyframe_metadata_file = f"{dir_path}/keyframes_metadata.json"

def preprocess_object_detection(data):
    organized_data = {'objects': dict(), 'counts': dict()}
    for item in data:
        label = item['label']
        if label not in organized_data['objects']:
            organized_data['objects'][label] = []
            organized_data['counts'][label] = 0
        organized_data['objects'][label].append({
            "score": item['score'],
            "box": item['box']
        })
        organized_data['counts'][label] += 1

    return organized_data

def combine_object_extraction_metadata_json_files(directory, output_file, keyframe_metadata_file):
    combined_data = {}

    for root, _, files in os.walk(directory):

        for file in files:
            if file.endswith('_detection.json'):
                file_path = os.path.join(root, file)

                parts = root.split(os.sep)
                video_name = parts[-1]  # L01_V001, L01_V002, etc.
                frame_id = file.split('_')[0]  # 000139, etc.
                with open(keyframe_metadata_file, "r") as file:
                    keyframe_metadata_dict = json.load(file)
                    
                # Fix frame_id
                for keyframe_key, keyframe_value in keyframe_metadata_dict.items():
                    if (video_name == f"{keyframe_value['video_path'].split('.')[0].split('/')[-1]}") and (frame_id == f"{keyframe_value['frame_path'].split('.')[0].split('/')[-1]}"):
                        frame_id = keyframe_key.split("_")[-1]
                
                key = f"{video_name}_{frame_id}_detection"

                with open(file_path, 'r') as f:
                    data = json.load(f)

                combined_data[key] = preprocess_object_detection(data)
                
    keys = list(combined_data.keys())
    keys.sort()
    sorted_combined_data = {i : combined_data[i] for i in keys}
    with open(output_file, 'w') as f:
        json.dump(sorted_combined_data, f)

    print(f'Combined object detection metadata successful: {output_file}')
    

combine_object_extraction_metadata_json_files(object_extraction_dir, combined_object_extraction_filename, keyframe_metadata_file)

Combined object detection metadata successful: object_extraction_metadata.json


# Combine OCR

In [ ]:
keyframe_metadata_file = f"{dir_path}/keyframes_metadata.json"

def combine_ocr_metadata_json_file(directory, output_file, keyframe_metadata_file):
    combined_data = {}
    if os.path.exists(output_file):
        with open(output_file, 'r') as existing_file:
            combined_data = json.load(existing_file)
    for filename in os.listdir(directory):
        video_id = f"{filename.split('.')[0].split('_')[0]}_{filename.split('.')[0].split('_')[1]}"
        file_name = filename.split('.')[0]
        if filename.endswith('.json'):
            file_path = os.path.join(directory, filename)

        with open(file_path, 'r') as file: # OCR json
                data = json.load(file)
        for key, item in data.items():
            # fix frame id
            with open(keyframe_metadata_file, "r") as file:
                keyframe_metadata_dict = json.load(file)
            for keyframe_key, keyframe_value in keyframe_metadata_dict.items():
                keyframe_img_name = keyframe_value["frame_path"].split("/")[-1]
                keyframe_video_name = keyframe_value['video_path'].split('.')[0].split('/')[-1]
                if (key == keyframe_img_name) and (video_id == keyframe_video_name):
                    key_name = keyframe_key
                    
            combined_data[f"{key_name}_ocr"] = item
    keys = list(combined_data.keys())
    keys.sort()
    sorted_combined_data = {i : combined_data[i] for i in keys}
    with open(output_file, 'w') as outfile:
        json.dump(sorted_combined_data, outfile)
    print(f'Combined multi tag metadata successful: {output_file}')
    
combine_ocr_metadata_json_file(ocr_metadata_dir, combined_ocr_metadata_filename, keyframe_metadata_file)

Combined multi tag metadata successful: ocr_metadata.json


# Combine tag

In [ ]:
keyframe_metadata_file = f"{dir_path}/keyframes_metadata.json"

def combine_multi_tag_metadata_json_file(directory, output_file, keyframe_metadata_file):
    combined_data = {}
    if os.path.exists(output_file):
        with open(output_file, 'r') as existing_file:
            combined_data = json.load(existing_file)
    for filename in os.listdir(directory):
        video_id = f"{filename.split('_')[0]}_{filename.split('_')[1]}"
        if filename.endswith('.json'):
            file_path = os.path.join(directory, filename)

        with open(file_path, 'r') as file:
                data = json.load(file)
        for key, item in data.items():
            # image name = key
            with open(keyframe_metadata_file, "r") as file:
                keyframe_data = json.load(file)
            for keyframe_key, keyframe_value in keyframe_data.items():
                if (key == keyframe_value["frame_path"].split("/")[-1]) and (video_id == keyframe_value['video_path'].split('.')[0].split('/')[-1]):
                    key_name = f"{keyframe_key}_tag"
        
            combined_data[key_name] = item
    keys = list(combined_data.keys())
    keys.sort()
    sorted_combined_data = {i : combined_data[i] for i in keys}
    with open(output_file, 'w') as outfile:
        json.dump(sorted_combined_data, outfile)
    print(f'Combined multi tag metadata successful: {output_file}') 
combine_multi_tag_metadata_json_file(tag_metadata_dir, combined_tag_metadata_filename, keyframe_metadata_file)

Combined multi tag metadata successful: tag_metadata.json


# Combine Audio

In [ ]:
keyframe_metadata_file = f"{dir_path}/keyframes_metadata.json"

def combine_audio_tag_metadata_json_file(audio_directory, output_file, keyframe_metadata_file):
    combined_data = {}

    if os.path.exists(output_file):
        with open(output_file, 'r') as existing_file:
            combined_data = json.load(existing_file)

    with open(keyframe_metadata_file, "r") as keyframe_file:
        keyframe_metadata_dict = json.load(keyframe_file)
        
    for sub_dir in (os.listdir(audio_directory)):
        sub_dir_path = f'{audio_directory}/{sub_dir}'
        for audio_json in (os.listdir(sub_dir_path)):
            video_id = f"{sub_dir}_{audio_json.split('.')[0]}"
            audio_json_path = f"{sub_dir_path}/{audio_json}"
            with open(audio_json_path, "r") as audio_file:
                audio_list = json.load(audio_file)
            for item in audio_list:
                for keyframe_key, keyframe_value in keyframe_metadata_dict.items():
                    frame_idx = keyframe_key.split("_")[-1]
                    start, end = item['segment_id']
                    video_id_keyframe = keyframe_value['video_path'].split(".")[0].split('/')[-1]
                    if (video_id == video_id_keyframe) and (int(frame_idx) >= start) and (int(frame_idx) <= end):
                        combined_data[f"{keyframe_key}_audio"] = item['transcription']
                        
                        
    fill_empty_audio_combined_data = {}
    keys_combined_data = combined_data.keys() # Taking all keys in combined data dict
    for key_keyframe in keyframe_metadata_dict.keys():
        if f"{key_keyframe}_audio" not in keys_combined_data:
            fill_empty_audio_combined_data[f"{key_keyframe}_audio"] = {"transcription" : ""}
    combined_data.update(fill_empty_audio_combined_data)
    
    
    keys = list(combined_data.keys())
    keys.sort()
    sorted_combined_data = {i : combined_data[i] for i in keys}
    with open(output_file, 'w') as outfile:
        json.dump(sorted_combined_data, outfile)
    
combine_audio_tag_metadata_json_file(audio_metadata_dir, combined_audio_metadata_filename, keyframe_metadata_file)

# Combined final file

In [ ]:
# def combined_json_file(json_files, output_file):
#     combined_data = {}
    
#     for file_name, data in json_files.items():
#         path = data['path']
#         key_ext = data.get('key_extension', '')
#         with open(path, 'r') as f:
#             json_data = json.load(f)
#         print(f"Preprocessing metadata file: {file_name}")
#         for key, data in json_data.items():
#             if key_ext:
#                 key = key.replace(f'_{key_ext}', '')
#             if key not in combined_data:
#                 combined_data[key] = {}
                
#             combined_data[key][key_ext] = data
            
#     with open(output_file, 'w') as f:
#         json.dump(combined_data, f)

#     print(f'Combined final metadata successful: {output_file}')

In [ ]:
# json_files = {
#     'keyframe_metadata': {
#         'path': combined_keyframes_metadata_filename,
#         'key_extension': 'keyframe',
#     },
#     'object_extraction': {
#         'path': combined_object_extraction_filename,
#         'key_extension': 'detection'
#     }
# }

# # combined_json_file(json_files, final_metadata_filename)